# Julia Types and Multiple Dispatch

All objects and variables in Julia have a type

This is similar to other dynamic languages, but in Julia types are extra important because...

1. They form the basis of Julia's main super power: *multiple dispatch*
2. They allow Julia to generate efficient machine code for a function, depending on the types

> Note parts 1 and 2 are related

In this notebook we will do a quick, high-level overview of Julia's type system

Then we will learn about multiple dispatch and see what it lets us do!

## Types intro

All objects and variables in Julia have a type

Use the `typeof` function to see the type:

In [49]:
x = 10.0
@show typeof(1)
@show typeof(x)
@show typeof([1, 2, 3])
@show typeof((1, "hi", [1, 2.0]))

typeof(1) = Int64
typeof(x) = Float64
typeof([1, 2, 3]) = Array{Int64,1}
typeof((1, "hi", [1, 2.0])) = Tuple{Int64,String,Array{Float64,1}}


Tuple{Int64,String,Array{Float64,1}}

## Abstract vs. Concrete Types

Julia types come in two different flavors:

1. Abstract types
2. Concrete types

Abstract types are used to organize a hierarchy of types

All objects or variables have exactly one concrete type

### `Any`

There is one type called `Any` that is the "top" of the tree of types

All types are subtypes of `Any` (including `Any` itself)


In [50]:
supertypes(Any)

(Any,)

In [51]:
subtypes(Any)

563-element Array{Any,1}:
 AbstractArray
 AbstractChannel
 AbstractChar
 AbstractDict
 AbstractDisplay
 AbstractLayout
 AbstractRNG
 AbstractSet
 AbstractString
 AbstractTrace
 AbstractTrees.AbstractShadowTree
 AbstractTrees.ImplicitChildStates
 AbstractTrees.ImplicitRootState
 ⋮
 WebIO.Node
 WebIO.ObservableNode
 WebIO.Scope
 WebIO.Sync
 WebIO.SyncCallback
 WebIO.WebIOServer
 WebSockets.ServerWS
 WebSockets.WebSocketFragment
 Widgets.AbstractBackend
 ZMQ.Context
 ZMQ.Socket
 ZMQ._Message

### Example: Number

To understand how these types work, let's look at how the Julia type system represents numbers

The "top" of the tree of numeric types is `Number`

It's only super type (parent type) is `Any`

In [97]:
supertype(Number)

Any

Number has two subtypes:

`Real` and `Complex`

In [53]:
subtypes(Number)

2-element Array{Any,1}:
 Complex
 Real

We can continue to traverse down the branch for `Real` numbers

In [54]:
subtypes(Real)

5-element Array{Any,1}:
 AbstractFloat
 AbstractIrrational
 FixedPointNumbers.FixedPoint
 Integer
 Rational

In [55]:
subtypes(Integer)

3-element Array{Any,1}:
 Bool
 Signed
 Unsigned

In [56]:
subtypes(Signed)

6-element Array{Any,1}:
 BigInt
 Int128
 Int16
 Int32
 Int64
 Int8

Eventually we reach the bottom of the tree and there are no more subtypes

In [98]:
subtypes(Int64)

Type[]

In the tree, all nodes except the leaf nodes are abstract

All leaf nodes are concrete

In [99]:
for T in [Number, Real, Integer, Signed, Int64]
    @show T, isabstracttype(T), isconcretetype(T)
end

(T, isabstracttype(T), isconcretetype(T)) = (Number, true, false)
(T, isabstracttype(T), isconcretetype(T)) = (Real, true, false)
(T, isabstracttype(T), isconcretetype(T)) = (Integer, true, false)
(T, isabstracttype(T), isconcretetype(T)) = (Signed, true, false)
(T, isabstracttype(T), isconcretetype(T)) = (Int64, false, true)


We can print out one branch of the tree using `Base.show_supertypes`

In [101]:
Base.show_supertypes(Signed)

Signed <: Integer <: Real <: Number <: Any

In [59]:
Base.show_supertypes(Int64)

Int64 <: Signed <: Integer <: Real <: Number <: Any

Using the `AbstractTrees` package we can print out the tree for any abstract type:

In [60]:
using AbstractTrees
AbstractTrees.children(x::Type) = subtypes(x)
print_tree(Number)

Number
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ FixedPoint
   │  ├─ Fixed
   │  └─ Normed
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


Question: based on the printout above, is `Complex` an abstract or concrete type?

Abstract types are defined using the syntax 

```julia
abstract type NAME <: PARENT end
```

where `NAME` is the name of the type and `PARENT` is the supertype

There can only be one supertype for each type

Let's see how the `Real` branch of numbers is defined in Julia

https://github.com/JuliaLang/julia/blob/676ccf4eaa9b6e6c6a53f75abb4bf3e1a2457426/base/boot.jl#L206-L230

### Concrete Types

Concrete types are types we can create

Every object or variable has exactly one concrete type

This is what is shown by `typeof`

Typically we create an instance of concrete type `T` using `T(data)`

> It is actually more sophisticated than that, but we'll get to that later...

In [62]:
@show typeof(1)
@show typeof(Int32(1));

typeof(1) = Int64
typeof(Int32(1)) = Int32


There are a few flavors of concrete type:

- `primitive type`
- composite type
    - `struct`
    - `mutable struct`

A primitive type is a type whose only structure is the number of bits that type occupies in memory

We saw this in the Julia code for defining the `Real` branch of `Number`'s subtypes:

```julia
primitive type Float32 <: AbstractFloat 32 end
primitive type Float64 <: AbstractFloat 64 end
```

I've been programming Julia for ~7 years and I've never had to create a primitive type...

### Composite Types

Composite types have one or more named and typed fields


They are defined with the syntax

```julia
struct NAME <: SUPERTYPE
    name1::TYPE1
    name2::TYPE2 
end
```

Note that  the field types are optional (but very helpful and strongly encouraged)

Also note that `SUPERTYPE` is optional -- if not given it will implicitly be `ANY`

Let's see an example

In [102]:
struct Address
    number::Int
    street::String
    unit::String
    city::String
    state::String
    zip::Int 
end

In [103]:
addr = Address(123, "5th avenue", "", "New York", "NY", 10023)
addr

Address(123, "5th avenue", "", "New York", "NY", 10023)

In [104]:
dump(addr)

Address
  number: Int64 123
  street: String "5th avenue"
  unit: String ""
  city: String "New York"
  state: String "NY"
  zip: Int64 10023


Field types can be built in, or types we create

In [105]:
using Dates

abstract type Person end

struct Friend <: Person
    name::String
    birthday::Date
    address::Address
end

struct Stranger <: Person
end

In [106]:
jim = Friend("Jim Halpert", Date(1979, 10, 20), addr)
jim

Friend("Jim Halpert", Date("1979-10-20"), Address(123, "5th avenue", "", "New York", "NY", 10023))

In [73]:
dump(jim)

Friend
  name: String "Jim Halpert"
  birthday: Date
    instant: Dates.UTInstant{Day}
      periods: Day
        value: Int64 722742
  address: Address
    number: Int64 123
    street: String "5th avenue"
    unit: String ""
    city: String "New York"
    state: String "NY"
    zip: Int64 10023


When defined with `struct`, an instance's fields cannot be changed after being created

In [107]:
jim.name = "Dwight Schrute"

LoadError: setfield! immutable struct of type Friend cannot be changed

To be able to change a field we need to use `mutable struct` instead

In [110]:
mutable struct Enemy2 <: Person
    name::String
end

dwight = Enemy2("Dwight")

Enemy2("Dwight")

In [111]:
dwight.name = "Dwight Schrute"

dump(dwight)

Enemy2
  name: String "Dwight Schrute"


### Parametric Types

Julia has a very powerful concept called parametric types

We won't have time to cover them in detail, but will look at `Array` as an example

See the section from the [docs](https://docs.julialang.org/en/v1/manual/types/#Parametric-Types) for more information

In [80]:
x = [1, 2]
typeof(x)

Array{Int64,1}

`Array` is the name of the type and `Int64`, `1` are the type parameters

Type parameters allow the same type/struct to represent data of different underlying types

In [81]:
typeof([1.0, 2.0])

Array{Float64,1}

In [82]:
typeof(["A", "b"])

Array{String,1}

In [85]:
m = [1 2; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [86]:
typeof(m)

Array{Int64,2}

### Multiple Dispatch

Now we are ready to talk about Julia's core abstraction (and a primary source of its extreme power!): multiple dispatch


Suppose we are trying to decide who to invite to a dinner party

Our rule is that friends should get a definite yes

Enemies a definite no

Strangers a 50%/50% toss up 

```python
class Person:
    def should_invite_to_dinner(self):
        return random.rand() < 0.5

class Friend(Person):
    def dinner(self):
        return True
    
class Enemy(Person):
    def dinner(self, wife_says, age):
        return False

def should_invite_to_dinner(x):
    if isinstance(x, Friend):
        return True
    elif isinstance(x, Enemy):
        return False
    else:
        return random.random() > 0.5
    
```   

In [127]:
should_invite_to_dinner(x) = "fallback"
should_invite_to_dinner(x::Friend) = true
should_invite_to_dinner(x::Enemy2) = false
should_invite_to_dinner(x::Person) = rand() < 0.5

should_invite_to_dinner (generic function with 7 methods)

In [128]:
should_invite_to_dinner(10)

"fallback"

In [119]:
should_invite_to_dinner(x::Person, wife_says::Bool) = wife_says

should_invite_to_dinner (generic function with 4 methods)

In [120]:
should_invite_to_dinner(x::Person, age::Int) = age > 21

should_invite_to_dinner (generic function with 5 methods)

In [123]:
@which should_invite_to_dinner(dwight, true)

should_invite_to_dinner(x::Person, wife_says::Bool) in Main at In[119]:1

In [126]:
@which should_invite_to_dinner(dwight, 30)

should_invite_to_dinner(x::Person, age::Int64) in Main at In[120]:1

In [118]:
@show should_invite_to_dinner(jim)
@show should_invite_to_dinner(dwight)

should_invite_to_dinner(jim) = true
should_invite_to_dinner(dwight) = false


false

In [96]:
creed = Stranger()

for _ in 1:10
    @show should_invite_to_dinner(creed)
end

should_invite_to_dinner(creed) = false
should_invite_to_dinner(creed) = false
should_invite_to_dinner(creed) = false
should_invite_to_dinner(creed) = true
should_invite_to_dinner(creed) = false
should_invite_to_dinner(creed) = true
should_invite_to_dinner(creed) = true
should_invite_to_dinner(creed) = false
should_invite_to_dinner(creed) = false
should_invite_to_dinner(creed) = false


Notice that for the invitation for a our `Stranger` `creed` is only sent about 1/2 the time

This happened even though we didn't define a method for an argument of type `Stranger`

Because `Stranger` is a subtype of `Person`, when we asked Julia `should_invite_to_dinner(creed)` the *most specific* method of the `should_invite_to_dinner` function was the one defined for `::Person`